In [2]:
from read_edf import plot_signals
from read_result import read_result_file
from read_result import plot_sleep_stages
from split_signal import split_edf_by_annotations
from split_signal import split_edf_by_30s
from sklearn.decomposition import PCA
import processing_data
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
import svm_builder
import importlib

In [3]:
feature_number = 105

X = np.empty((0,feature_number))
y = np.empty((0,))
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X = np.concatenate((X, file['features']))
        y = np.concatenate((y, file['labels']))
print(X.shape)
print(y.shape)

(393345, 105)
(393345,)


下面这段是 Balance with Over sample

In [4]:
import balance
importlib.reload(balance)
balance_x, balance_y = balance.get_balance_class_oversample(X, y)
print(balance_x.shape)
print(balance_y.shape)

(1968512, 105)
(1968512,)


In [5]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(balance_x)
X_scaled = scaler.fit_transform(balance_x)

In [6]:
pca1 = PCA(n_components= 0.999).fit(X_scaled)
pca2 = PCA(n_components= 0.95).fit(X_scaled)
X_pca1 = pca1.transform(X_scaled)
X_pca2 = pca2.transform(X_scaled)
print(X_pca1.shape)
print(X_pca2.shape)

(1968512, 81)
(1968512, 43)


In [7]:
model_X = X_pca1
model_y = balance_y
print(model_X.shape, model_y.shape)

(1968512, 81) (1968512,)


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
# devide X and y into train dataset and test dataset:
X_train, X_test, y_train, y_test = train_test_split(model_X, model_y, test_size=0.3, random_state=42)

In [10]:
import logistic_builder

In [19]:
importlib.reload(logistic_builder)
log_model = logistic_builder.train_logistic_model(X_train, y_train)

D:\Anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\Anaconda3\Lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


Training set score of logistic: 0.7289206589464735
Testing set score of logistic: 0.7290342245057911


In [23]:
y_pred = log_model.predict(X_test)
y_pred = y_pred.astype(int).tolist()
# y_test = y_test.astype(int).tolist()


In [24]:
import scoring
importlib.reload(scoring)

<module 'scoring' from 'E:\\2023 Fall\\IML\\project\\iml-project\\scoring.py'>

In [25]:
scoring.score(y_pred, y_test)

Class 0: Precision: 0.8176691256073919, Recall: 0.7704552463644343
Class 1: Precision: 0.5807565627765214, Recall: 0.5599597108662164
Class 2: Precision: 0.5995085995085995, Recall: 0.6245437739963028
Class 3: Precision: 0.7055396625010134, Recall: 0.7202378841083484
Class 4: Precision: 0.8115595262921053, Recall: 0.8751464653039969
Class 5: Precision: 0.7289109325516266, Recall: 0.6616026036964888
Class 6: Precision: 0.8553029441855178, Recall: 0.8934712392533131
mf1 score: 0.7289043247851748
acc_score: 0.729335505305188
kappa_score: 0.8992207916025664


E:\2023 Fall\IML\project\iml-project\scoring.py:46: RuntimeWarning: overflow encountered in scalar add
  pe += np.dot(cfm[:,i], cfm[i,:])
